In [4]:
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
import os
import mysql.connector
load_dotenv()

db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_port = os.getenv("db_port")
db_name = "jiohotstar_ads"

db_path = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"

db = SQLDatabase.from_uri(db_path)
print(db.get_usable_table_names())

['matches']


In [ ]:
db.


CREATE TABLE matches (
	match_id INTEGER NOT NULL, 
	teams VARCHAR(255), 
	location VARCHAR(255), 
	match_type VARCHAR(100), 
	winner VARCHAR(255), 
	video_path VARCHAR(255), 
	match_timestamp DATETIME, 
	created_at TIMESTAMP NULL DEFAULT CURRENT_TIMESTAMP, 
	PRIMARY KEY (match_id)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from matches table:
match_id	teams	location	match_type	winner	video_path	match_timestamp	created_at
1	IN Vs Aus	IN	T20	IN	data/1/source_video.mp4	2025-11-08 02:00:00	2025-11-19 08:46:08
2	NZ vs AUS	NZ	T50	NZ	data/2/source_video.mp4	2025-11-11 02:00:00	2025-11-23 07:53:34
*/


In [8]:
print(db.get_table_info())


CREATE TABLE matches (
	match_id INTEGER NOT NULL, 
	teams VARCHAR(255), 
	location VARCHAR(255), 
	match_type VARCHAR(100), 
	winner VARCHAR(255), 
	video_path VARCHAR(255), 
	match_timestamp DATETIME, 
	created_at TIMESTAMP NULL DEFAULT CURRENT_TIMESTAMP, 
	PRIMARY KEY (match_id)
)COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB DEFAULT CHARSET=utf8mb4

/*
3 rows from matches table:
match_id	teams	location	match_type	winner	video_path	match_timestamp	created_at
1	IN Vs Aus	IN	T20	IN	data/1/source_video.mp4	2025-11-08 02:00:00	2025-11-19 08:46:08
2	NZ vs AUS	NZ	T50	NZ	data/2/source_video.mp4	2025-11-11 02:00:00	2025-11-23 07:53:34
*/


In [6]:
db.run("SELECT * FROM matches")

"[(1, 'IN Vs Aus', 'IN', 'T20', 'IN', 'data/1/source_video.mp4', datetime.datetime(2025, 11, 8, 2, 0), datetime.datetime(2025, 11, 19, 8, 46, 8)), (2, 'NZ vs AUS', 'NZ', 'T50', 'NZ', 'data/2/source_video.mp4', datetime.datetime(2025, 11, 11, 2, 0), datetime.datetime(2025, 11, 23, 7, 53, 34))]"

In [14]:
import ollama

model_name = "llama3.2:1b"  # replace with your actual model name
messages = [{"role": "system", 
    "content": f"You are a sql query generator. Generate a sql query based on the user input. only return the sql query. Tabel info is given below: {db.get_table_info()}"}]

while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    messages.append({"role": "user", "content": user_input})
    
    response = ollama.chat(model=model_name, messages=messages)
    bot_reply = response.message.content
    result = db.run(bot_reply)
    print("Bot:", result)
    
    messages.append({"role": "assistant", "content": bot_reply})


Bot: [(1, 'IN Vs Aus', 'IN', 'T20', 'IN', 'data/1/source_video.mp4', datetime.datetime(2025, 11, 8, 2, 0), datetime.datetime(2025, 11, 19, 8, 46, 8)), (2, 'NZ vs AUS', 'NZ', 'T50', 'NZ', 'data/2/source_video.mp4', datetime.datetime(2025, 11, 11, 2, 0), datetime.datetime(2025, 11, 23, 7, 53, 34))]
Bot: [(1, 'IN Vs Aus', 'IN', 'T20', 'IN', 'data/1/source_video.mp4', datetime.datetime(2025, 11, 8, 2, 0), datetime.datetime(2025, 11, 19, 8, 46, 8)), (2, 'NZ vs AUS', 'NZ', 'T50', 'NZ', 'data/2/source_video.mp4', datetime.datetime(2025, 11, 11, 2, 0), datetime.datetime(2025, 11, 23, 7, 53, 34))]
Bot: [(1, 'IN Vs Aus', 'IN', 'T20', 'IN', 'data/1/source_video.mp4', datetime.datetime(2025, 11, 8, 2, 0), datetime.datetime(2025, 11, 19, 8, 46, 8)), (2, 'NZ vs AUS', 'NZ', 'T50', 'NZ', 'data/2/source_video.mp4', datetime.datetime(2025, 11, 11, 2, 0), datetime.datetime(2025, 11, 23, 7, 53, 34))]


ProgrammingError: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Here are the column names in the Matches table:

match_id, teams, location, matc' at line 1
[SQL: Here are the column names in the Matches table:

match_id, teams, location, match_type, winner, video_path, match_timestamp, created_at]
(Background on this error at: https://sqlalche.me/e/20/f405)

Usable tables: ['matches']


In [19]:

# Interactive loop: user inputs NL question → generate SQL → execute → print results
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    messages.append({"role": "user", "content": user_input})

    # Generate SQL query with Ollama LLM
    response = ollama.chat(model=model_name, messages=messages)
    sql_query = response.message.content.strip()

    print("Generated SQL:", sql_query)

    # Run generated SQL against DB safely (you can add validation here)
    try:
        result = db.run(sql_query)
        print("Result:", generate_natural_language_answer(sql_query,result))
    except Exception as e:
        print("SQL execution error:", e)

    # Append the assistant message (SQL query generated) to keep context
    messages.append({"role": "assistant", "content": sql_query})


Generated SQL: SELECT * FROM matches;
SQL execution error: generate_natural_language_answer() missing 3 required positional arguments: 'sql', 'result', and 'llm'


In [20]:
from langchain_community.utilities import SQLDatabase
from dotenv import load_dotenv
import os
import json
import ollama

load_dotenv()

# Load DB credentials from environment
db_user = os.getenv("db_user")
db_password = os.getenv("db_password")
db_host = os.getenv("db_host")
db_port = os.getenv("db_port")
db_name = "jiohotstar_ads"

# Build DB URI and initialize LangChain SQLDatabase
db_path = f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
db = SQLDatabase.from_uri(db_path)

# Get table info as schema context
schema_info = db.get_table_info()

# Initialize Ollama client and model name
model_name = "llama3.2:1b"

# Initialize chat messages with system role and schema information
messages = [
    {
        "role": "system",
        "content": (
            "You are a SQL query generator. "
            "Generate a valid SQL SELECT query based on the user input. "
            "Only return the SQL query. "
            "Table info is given below:\n" + schema_info
        ),
    }
]

print("Usable tables:", db.get_usable_table_names())

answer_prompt_template = """
You are a helpful data analyst.

Original question:
{question}

SQL query executed:
{sql}

Query result (showing first few rows):
{result}

Using ONLY the information above, answer the question clearly and concisely.
If the result does not provide enough info, say you cannot answer confidently.
"""

def generate_natural_language_answer(question, sql, result, llm, max_rows=10):
    # Convert result rows to short JSON string for prompt
    trimmed_result = json.dumps(result[:max_rows], indent=2, default=str)

    prompt = answer_prompt_template.format(question=question, sql=sql, result=trimmed_result)
    messages = [
        {"role": "system", "content": "You are a helpful data analyst."},
        {"role": "user", "content": prompt}
    ]
    response = llm.chat(model=model_name, messages=messages)
    return response.message.content.strip()

Usable tables: ['matches']


In [21]:

# Interactive loop: user inputs NL question → generate SQL → execute → print NL results
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    messages.append({"role": "user", "content": user_input})

    # Generate SQL query with Ollama LLM
    response = ollama.chat(model=model_name, messages=messages)
    sql_query = response.message.content.strip()
    print("Generated SQL:", sql_query)

    # Run generated SQL against DB safely
    try:
        result = db.run(sql_query)
        # Generate natural language answer from SQL result
        nl_answer = generate_natural_language_answer(user_input, sql_query, result, ollama)
        print("Bot (natural language answer):", nl_answer)
    except Exception as e:
        print("SQL execution error:", e)

    # Append the assistant message (SQL query generated) to keep context
    messages.append({"role": "assistant", "content": sql_query})


Generated SQL: SELECT * FROM matches
Bot (natural language answer): I can help you with that.

You should see a table with one or more columns. Here is an example of what it might look like:
```
+----------+------------+
| MatchID | Player1   | ...       |
+----------+------------+
```

This shows all records in the `matches` table, including all columns (`*`).
Generated SQL: SELECT * FROM matches LIMIT 1
Bot (natural language answer): I can help with that. Since we are dealing with a database table called "matches", the query `SELECT * FROM matches LIMIT 1` will return only one row from this table.


Given the provided SQL query results, it seems like they are displaying the first few rows of the "matches" table. To confirm if there is indeed just one row to be displayed in the "matches" table, we would need more information about the data type or specific characteristics that might influence how many rows are returned.

However, assuming the result shows a single match record with re